In [ ]:
import os
import pandas as pd
from itertools import islice
import torch
from torch.utils.data import DataLoader
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parents[1]))
from utils.text_metrics import evaluate_all_metrics, save_metrics_to_json
from utils.train_comparison import *
from utils.processing import image_transform
from utils.models.gpt_models import DinoGPTCaptioner, DinoGPT2Captioner
from utils.data.chexpert_dataset import CheXpertDataset
from utils.data.padchest_dataset import PadChestGRDataset

# Data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

CSV_PATH = "../../../Datasets/CheXpertPlus/df_chexpert_plus_240401.csv"
IMG_ROOT = "../../../Datasets/CheXpertPlus/PNG/"

TEXT_COL = "section_impression"
PATH_COL = "path_to_image"

IMG_SIZE = 224
MAX_LEN = 64
NUM_BATCH = 8

tf = image_transform(img_size=IMG_SIZE)

ds_train = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="train", transform=tf, text_col=TEXT_COL)
ds_valid = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="valid", transform=tf, text_col=TEXT_COL)
ds_test = CheXpertDataset(img_root=IMG_ROOT, csv_path=CSV_PATH, split="test", transform=tf, text_col=TEXT_COL)

#labels = pd.read_csv(CSV_PATH)[TEXT_COL].tolist()

tokenizer = build_tokenizer_from_labels(captions=None)
pad_id = tokenizer.pad_token_id
eos_id = tokenizer.eos_token_id
bos_id = tokenizer.bos_token_id
collate_fn = CaptionCollate(tokenizer, pad_id)

train_loader = DataLoader(ds_train, batch_size=NUM_BATCH, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(ds_valid, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(ds_test, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)

Using device: cuda
[INFO] Kept 95718/223462 rows with existing PNGs
[INFO] Kept 95718/223462 rows with existing PNGs
[INFO] Kept 95718/223462 rows with existing PNGs


In [ ]:
tokenizer_size = tokenizer.vocab_size
print("Tokenizer size:", tokenizer_size)

Tokenizer size: 58996


# Model

In [ ]:
# DINO ViT-S/16 hidden size is 384 
EMBEDDING_D_IMG = 384
N_PREFIX = (IMG_SIZE // 16) ** 2  # number of visual prefix tokens (including CLS)

def pick_heads(d_model, target_head_dim=64):
    h = max(1, round(d_model / target_head_dim))
    while d_model % h != 0: h -= 1
    return h

D_MODEL = 768
N_HEAD = pick_heads(D_MODEL, 64)  # -> 12


model = DinoGPTCaptioner(
    vocab_size=tokenizer.vocab_size,
    d_img=EMBEDDING_D_IMG,
    pad_id=pad_id,
    d_model=D_MODEL,
    n_layer=12,
    n_head=N_HEAD,
    n_prefix=N_PREFIX,           # number of visual prefix tokens
    max_seq_len=512,
    dino_model_id="facebook/dinov3-vits16-pretrain-lvd1689m",
    freeze_dino=True,
).to(device)

# Print model parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params / 1_000_000:.2f} Millions")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable model parameters: {trainable_params / 1_000_000:.2f} Millions")

# Print model footprint
model_footprint_in_gb = (total_params * 4) * (1e-9)  # assuming 4 bytes per parameter (float32)
print(f"Approximate model footprint: {model_footprint_in_gb:.2f} GB")

# after model init
model.decoder.lm_head.weight = model.decoder.tok_emb.weight  # weight tying

Total model parameters: 198.08 Millions
Trainable model parameters: 176.48 Millions
Approximate model footprint: 0.79 GB


# Train Parameters

In [ ]:
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=3e-4, weight_decay=1e-2
)
loss = sequence_ce_loss
NUM_EPOCHS = 10
BATCHES_PER_EPOCH = 10

# Training

In [ ]:
import time
time_start = time.time()
for epoch in range(NUM_EPOCHS):
    slice_train_loader = islice(train_loader, BATCHES_PER_EPOCH)
    slice_valid_loader = islice(valid_loader, BATCHES_PER_EPOCH)
    train_stats = train_one_epoch(model, slice_train_loader, optimizer, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss, grad_clip=1.0)
    val_stats = evaluate(model, slice_valid_loader, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss)
    print(f"Epoch {epoch + 1}: Train Loss={train_stats['loss']:.4f}, PPL={train_stats['ppl']:.2f} | "
            f"Val Loss={val_stats['val_loss']:.4f}, Val PPL={val_stats['val_ppl']:.2f}")
training_time = time.time() - time_start

Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.35it/s]


Epoch 1: Train Loss=9.6063, PPL=20127.06 | Val Loss=8.4365, Val PPL=4641.53


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.39it/s]


Epoch 2: Train Loss=7.8496, PPL=2768.84 | Val Loss=7.4829, Val PPL=1813.88


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.38it/s]


...

Epoch 8: Train Loss=6.5352, PPL=698.08 | Val Loss=6.3059, Val PPL=569.68


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.35it/s]


Epoch 9: Train Loss=6.2286, PPL=518.75 | Val Loss=6.1103, Val PPL=471.75


Evaluating: 100%|██████████| 10/10 [00:04<00:00,  2.35it/s]

Epoch 10: Train Loss=6.1158, PPL=469.97 | Val Loss=5.9318, Val PPL=398.42


# Test Parameters

In [ ]:
BATCHES_PER_TEST = 1
GREEDY_DECODE = True
TEST_MAX_LEN = 256
TEST_TOP_P = 0.9
TEST_TEMPERATURE = 0.9

# Test

In [ ]:
slice_test_loader = islice(test_loader, BATCHES_PER_TEST)
test_stats = evaluate(model, slice_test_loader, device, pad_id, num_batches=BATCHES_PER_TEST)
print(f"Test Loss={test_stats['val_loss']:.4f}, Test PPL={test_stats['val_ppl']:.2f}")

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  2.06it/s]

Test Loss=5.7580, Test PPL=316.72


# Test Report Generation

In [ ]:
# capitalize first word and first word after each "."
def capitalize_sentences(s):
    parts = s.split('. ')
    parts = [p[:1].upper() + p[1:] if p else '' for p in parts]
    return '. '.join(parts)

generated_text = []
target_text = []
iteration = 0

with torch.no_grad():
    for pixel_values, ids_loader, paths, raw_labels in test_loader:
        iteration += 1
        pixel_values = pixel_values.to(device)
        # gen_ids = model.generate(
        #     pixel_values=pixel_values,
        #     bos_id=bos_id,
        #     eos_id=eos_id,
        #     max_new_tokens=TEST_MAX_LEN,
        #     beam_size=3,                # Set your desired beam size
        #     temperature=TEST_TEMPERATURE
        # )

        info = model.generate_with_logging(
            pixel_values=pixel_values,          # [B, C, H, W]
            bos_id=tokenizer.bos_token_id,
            eos_id=tokenizer.eos_token_id,
            tokenizer=tokenizer,
            preset="safe_sample",
            stop_sequences=["\n\n", "Impression:"],
            max_new_tokens=256,
        )

        # print("batch sequences shape:", info["sequences"].shape)
        # for i, s in enumerate(info["per_sample"]):
        #     print(f"[sample {i}] hit_eos={s['stopping']['hit_eos']} repetition={s['repetition']}")
        #     if "generated" in s["text"]:
        #         print(capitalize_sentences(s["text"]["generated"]))
        #         print("[Target text]", capitalize_sentences(raw_labels[i]))

        generated_text.extend([s["text"]["generated"] for s in info["per_sample"]])
        target_text.extend(raw_labels)

        # eval_results = evaluate_all_metrics(raw_labels, [s["text"]["generated"] for s in info["per_sample"]], evaluation_mode="CheXagent")
        # for metric, scores in eval_results.items():
        #     print(f"{metric}: {scores}")

        # save_metrics_to_json(eval_results, f"./results/gpt_model_results_{NUM_EPOCHS}_Chexpert.json")


        # print("Predictions (first batch):")
        # for i in range(gen_ids.size(0)):
        #     text_gen = tokenizer.decode(gen_ids[i].tolist())
        #     text_tgt = tokenizer.decode(ids_loader[i].tolist())
        #     print(f"\nGEN {i+1}:", capitalize_sentences(text_gen))
        #     print(f"TGT {i+1}:", capitalize_sentences(text_tgt))
        #     results = evaluate_all_metrics([text_tgt], [text_gen], evaluation_mode="CheXagent")
        #     for metric, scores in results.items():
        #         print(f"{metric}: {scores}")
        # del pixel_values, ids_loader, paths, raw_labels, info
        # torch.cuda.empty_cache()
        # break
        if iteration >= 1:  # Limit to 1 iteration for testing
            break

eval_results = evaluate_all_metrics(generated_text, target_text, evaluation_mode="CheXagent")
for metric, scores in eval_results.items():
    print(f"{metric}: {scores}")
eval_results["training_time_seconds"] = training_time
save_metrics_to_json(eval_results, f"./results/gpt_model_results_{NUM_EPOCHS}_Chexpert.json")

Using device: cuda:0
chexbert_f1_weighted: 0.36769738198309626
chexbert_f1_micro: 0.4
chexbert_f1_macro: 0.14009482580911153
chexbert_f1_micro_5: 0.4
chexbert_f1_macro_5: 0.1890909090909091
radgraph_f1_RG_E: 0.0944118605408928
radgraph_f1_RG_ER: 0.0913243006993007
